In [377]:
import http
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException

import re

In [378]:
df = pd.read_excel('data/rmi_data.xlsx')
df.head()
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11492 entries, 0 to 11491
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Tender Number        11096 non-null  object 
 1   Region               11096 non-null  object 
 2   City                 11096 non-null  object 
 3   Purpose              11096 non-null  object 
 4   Publication date     11096 non-null  object 
 5   Status               11096 non-null  object 
 6   Opening date         11096 non-null  object 
 7   Submission deadline  10972 non-null  object 
 8   Committee date       11096 non-null  object 
 9   Compound Number      11486 non-null  object 
 10  Units                11263 non-null  float64
 11  Winner Name          11024 non-null  object 
 12  Final Price          7985 non-null   object 
 13  Development Cost     10090 non-null  object 
 14  Valuation Price      10594 non-null  object 
 15  Minimum Price        8654 non-null  

In [379]:
df = df[~(df['Block'] == 'Not existing')]

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11492 entries, 0 to 11491
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Tender Number        11096 non-null  object 
 1   Region               11096 non-null  object 
 2   City                 11096 non-null  object 
 3   Purpose              11096 non-null  object 
 4   Publication date     11096 non-null  object 
 5   Status               11096 non-null  object 
 6   Opening date         11096 non-null  object 
 7   Submission deadline  10972 non-null  object 
 8   Committee date       11096 non-null  object 
 9   Compound Number      11486 non-null  object 
 10  Units                11263 non-null  float64
 11  Winner Name          11024 non-null  object 
 12  Final Price          7985 non-null   object 
 13  Development Cost     10090 non-null  object 
 14  Valuation Price      10594 non-null  object 
 15  Minimum Price        8654 non-null  

In [381]:
#adding X and Y cordinates to the file
if 'X' not in df.columns:
        df['X'] = None
if 'Y' not in df.columns:
        df['Y'] = None

In [382]:
def search_Block(block, driver):
    try:
        wait = WebDriverWait(driver, 10)
        search_bar = wait.until(EC.element_to_be_clickable((
            By.XPATH, '//*[contains(@class,"searchInput")]'
        )))
        # Clear existing content
        search_bar.click()  # Ensure the field is active
        search_bar.clear()  # Clear the field
        time.sleep(0.5)
        
        search_bar.send_keys(f'גוש {block}')
        time.sleep(1)
        
        # Check if autocomplete suggestions window appeared and has content
        try:
            autocomplete = driver.find_element(By.CLASS_NAME, "autocomplete-item")
            if autocomplete.is_displayed():
                # Click on the first suggestion
                autocomplete.click()
                
                # Check for second popup and click if it exists
                try:
                    time.sleep(1)  # Wait for second popup
                    result_popup = driver.find_element(By.CLASS_NAME, "result.resultSmall")
                    if result_popup.is_displayed():
                        print(f"Found second popup - clicking")
                        result_popup.click()
                except:
                    # No second popup found, that's fine
                    pass
                
                return 1  # Block exists and suggestion clicked
            else:
                # Block not found - remove focus from search field
                driver.find_element(By.TAG_NAME, "body").click()  # Click somewhere else
                return 0
        except:
            # Block not found - remove focus from search field  
            driver.find_element(By.TAG_NAME, "body").click()  # Click somewhere else
            return 0
        
    except TimeoutException:
        print("Search bar not found or not clickable within 10 seconds")
        return 0

In [383]:
def hover_on_marker_for_coordinates(driver):
    """Hover over the marker to update coordinates, don't click"""
    try:
        # חפש את התמונה של המרקר
        script = """
        var images = document.querySelectorAll('image');
        for (var i = 0; i < images.length; i++) {
            var href = images[i].getAttribute('href') || images[i].getAttribute('xlink:href');
            if (href && href.includes('marker.png')) {
                return images[i];
            }
        }
        return null;
        """
        
        marker = driver.execute_script(script)
        
        if marker:
            action = ActionChains(driver)
            action.move_to_element(marker).perform()  # Move to the image
            
            # print("Hovered over marker")
            time.sleep(1)  
            return True
        else:
            print("Marker not found")
            return False
            
    except Exception as e:
        print(f"Error hovering over marker: {e}")
        return False

In [384]:
def get_coordinates_from_screen(driver,flag):
    """Extract X and Y coordinates from the specific div"""
    print("get cordinates from screen")
    try:
        if flag==1:
            wait = WebDriverWait(driver, 5)
            # Find the specific div with coordinates
            element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "mapXY")))
            
            # Get the text content
            text = element.text
            
            # Extract X and Y using regex
            x_match = re.search(r'X:(\d+)', text)
            y_match = re.search(r'Y:(\d+)', text)
            
            if x_match and y_match:
                x_coordinate = x_match.group(1)
                y_coordinate = y_match.group(1)
                
                print(f"Extracted - X: {x_coordinate}, Y: {y_coordinate}")
                # FIX: השגיאה הייתה כאן - החזר כdictionary
                return {
                    'X': x_coordinate,
                    'Y': y_coordinate
                }
            else:
                print("Could not extract coordinates from text")
                return {'X': None, 'Y': None}
        else:
            return{
                    'X': 0,
                    'Y': 0
            }
    except Exception as e:
        print(f"Error extracting coordinates: {e}")
        return {'X': None, 'Y': None}

In [386]:
def main_function():
    """
    Main function for running the entire scraping process.
    """
    driver = webdriver.Chrome()
    driver.maximize_window()  # This line makes it full screen
    driver.get('https://www.govmap.gov.il/')
    time.sleep(2)    
    unique_blocks = df['Block'].dropna().unique()
    print(f"found {len(unique_blocks)} unique blocks")
    
    block_coordinates = {}
    for i, block in enumerate(unique_blocks):
        print(f"block {block}, ({i+1}/{len(unique_blocks)})")
        
        # Search for this block
        flag = search_Block(int(float(block)), driver)
        time.sleep(2)
        
        if flag == 1:
            hover_on_marker_for_coordinates(driver) 
            time.sleep(1)
        
        # Get coordinates
        coordinates = get_coordinates_from_screen(driver, flag)
        
        # Store in dictionary
        block_coordinates[block] = {
            'X': coordinates['X'],
            'Y': coordinates['Y']
        }
        
        print(f" block {block} -> X: {coordinates['X']}, Y: {coordinates['Y']}\n\n")
    
    # Final update of all rows
    print("\n updating coordinates for all the sections in the same block")
    for block, coords in block_coordinates.items():
        df.loc[df['Block'] == block, 'X'] = coords['X']
        df.loc[df['Block'] == block, 'Y'] = coords['Y']
    
    # Save final file
    df.to_csv('all_country_version_2.csv', index=False)
    print(f"\n done! total of {len(block_coordinates)} blocks")
    print("Sample results:")
    print(df[['Block', 'Section', 'X', 'Y']].head(10))
    
    driver.quit()

In [387]:
main_function()

found 417 unique blocks
block 11308.0, (1/417)
Found second popup - clicking
get cordinates from screen
Extracted - X: 196434, Y: 718774
 block 11308.0 -> X: 196434, Y: 718774


block 2909.0, (2/417)
Found second popup - clicking
get cordinates from screen
Extracted - X: 178293, Y: 615286
 block 2909.0 -> X: 178293, Y: 615286


block 5143.0, (3/417)
Found second popup - clicking
get cordinates from screen
Extracted - X: 190739, Y: 636559
 block 5143.0 -> X: 190739, Y: 636559


block 5142.0, (4/417)
Found second popup - clicking
get cordinates from screen
Extracted - X: 190872, Y: 637615
 block 5142.0 -> X: 190872, Y: 637615


block 2812.0, (5/417)
Found second popup - clicking
get cordinates from screen
Extracted - X: 178469, Y: 615830
 block 2812.0 -> X: 178469, Y: 615830


block 7792.0, (6/417)
Found second popup - clicking
get cordinates from screen
Extracted - X: 192992, Y: 684509
 block 7792.0 -> X: 192992, Y: 684509


block 2866.0, (7/417)
Found second popup - clicking
get cordin